In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-18"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9354,2024-03-18,Turquia Tkbl Feminina,12:00,Botas 2 F,Urla Bld F,1.13,5.40,132.5,1.84,1.86,0.0,0.00,0.00,ljzPlU4P,0.884956,0.185185,0.543478,0.537634,0.070141,123.116,91.215504,0.740891,0.6,1.341641,2.236068,79.35,1.990,1.371988,0.689441,-31.0,429.740,234.676315,0.546089,0.0,0.000000,NaN,420.90,6.572,3.926419,0.597447,-92.0,69,61,1.15,6.90,111.360,444.612,0.924761,0.007644,NaN,-0.94,-0.188,-0.691489,0.000000,0.0,0.000000,-1.900000e+00,-3.800000e-01,-1.157895e+01,0.000000,0.0,0.000000
9355,2024-03-18,Turquia Tbl,09:00,Esenler Erokspor,Fenerbahce 2,1.72,2.02,168.5,1.85,1.85,-2.5,1.98,2.43,6NzllXxG,0.581395,0.495050,0.540541,0.540541,0.076445,164.610,41.395828,0.251478,1.2,1.643168,1.369306,185.60,2.038,0.374927,0.183968,-17.0,216.980,52.745830,0.243091,1.8,1.643168,0.912871,199.10,2.460,1.149456,0.467259,21.0,80,110,2.32,1.81,151.804,258.322,0.113440,0.000000,0.144308,-0.48,-0.096,-7.500000,0.000000,0.0,0.000000,-2.660000e+00,-5.320000e-01,-1.917293e+00,0.371103,0.2,-0.171103
9356,2024-03-18,Turquia Superliga,14:30,Buyukcekmece,Bahcesehir Kol.,1.83,1.92,164.5,1.85,1.89,-2.5,2.04,2.15,KAfYMTYb,0.546448,0.520833,0.540541,0.529101,0.067281,255.178,126.726613,0.496620,0.6,1.341641,2.236068,373.23,3.112,1.808665,0.581191,-3.0,249.904,172.990288,0.692227,0.6,1.341641,2.236068,87.77,2.902,1.931546,0.665591,-22.0,87,67,4.29,1.31,222.340,249.322,0.033941,0.015125,0.037127,-1.30,-0.260,-3.192308,0.000000,0.0,0.000000,-1.800000e-01,-3.600000e-02,-2.555556e+01,0.000000,0.0,0.000000
9357,2024-03-18,Suécia Basketligan,15:04,Norrkoping,Jamtland,1.40,2.80,164.5,1.84,1.86,0.0,0.00,0.00,nwzdGaz3,0.714286,0.357143,0.543478,0.537634,0.071429,123.760,42.459681,0.343081,2.4,1.341641,0.559017,195.96,1.328,0.463487,0.349011,57.0,136.564,42.045898,0.307884,3.0,0.000000,0.000000,206.08,1.470,0.459293,0.312444,75.0,92,92,2.13,2.24,107.316,152.880,0.471405,0.007644,NaN,-0.76,-0.152,-2.631579,0.889047,0.8,-0.089047,2.220446e-16,4.440892e-17,4.053240e+16,0.611647,0.6,-0.011647
9358,2024-03-18,Lituânia Lkl,13:50,Nevezis,Juventus,3.86,1.23,175.5,1.84,1.86,0.0,0.00,0.00,bcVP4ypk,0.259067,0.813008,0.543478,0.537634,0.072075,292.212,194.548268,0.665778,1.2,1.643168,1.369306,129.60,3.872,3.313144,0.855667,-54.0,199.584,94.027178,0.471116,1.2,1.643168,1.369306,128.52,2.570,1.676395,0.652294,-3.0,72,119,1.80,1.08,258.744,204.092,0.730723,0.007644,NaN,3.50,0.700,4.085714,0.000000,0.0,0.000000,-1.160000e+00,-2.320000e-01,-9.913793e-01,0.000000,0.0,0.000000
9359,2024-03-18,Rússia Superliga,14:00,Bars Rostov,Dynamo Vladivostok,3.61,1.26,149.5,1.84,1.86,0.0,0.00,0.00,rsLQt22j,0.277008,0.793651,0.543478,0.537634,0.070659,189.088,117.320230,0.620453,1.8,1.643168,0.912871,98.28,2.488,1.578851,0.634586,-7.0,118.800,47.225415,0.397520,1.2,1.643168,1.369306,197.20,1.510,0.486210,0.321993,36.0,78,85,1.26,2.32,132.372,134.478,0.682423,0.007644,NaN,0.28,0.056,46.607143,0.000000,0.0,0.000000,-3.000000e-01,-6.000000e-02,-4.333333e+00,0.000000,0.0,0.000000
9360,2024-03-18,Rússia Liga Vtb United,14:00,Runa Basket Moscow,Astana,1.24,4.15,156.5,1.81,1.95,0.0,0.00,0.00,OjNCJAtk,0.806452,0.240964,0.552486,0.512821,0.047415,187.072,51.499197,0.275291,0.6,1.341641,2.236068,154.98,2.232,0.369959,0.165752,-25.0,164.776,72.298710,0.438770,0.6,1.341641,2.236068,245.70,2.450,1.116020,0.4

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9364,21:00,Uruguai Liga Uruguaia,Nacional,Hebraica y Macabi,1.69,Back Home
9372,14:00,Eslovênia Liga Nova Kbm,LTH Castings,Podcetrtek,1.64,Back Home
9406,11:00,Europa Liga Aba 2,Pelister,Borac Banja Luka,1.71,Back Home
